In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [5]:
#Prep Toronto Data

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0]
df = df[df.Borough != "Not assigned"]
df.reset_index(drop=True, inplace=True)
url = "http://cocl.us/Geospatial_data"
df2 = pd.read_csv(url)
new_df = pd.merge(left = df, right = df2, how = "outer", left_on= "Postal Code", right_on = "Postal Code")

In [7]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Canada")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
CLIENT_ID = '230YIWXJCKI42N4LNIXLR1IDXJHX5AFLOSOEBPJHPEVBRCU1' # your Foursquare ID
CLIENT_SECRET = 'MXMXV5RKVWDXJVT3Y0PGCXGF2GYIHSTIJJ1F32ENDGUQXNA0' # your Foursquare Secret
VERSION = '20180604'

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT =

Your credentials:
CLIENT_ID: 230YIWXJCKI42N4LNIXLR1IDXJHX5AFLOSOEBPJHPEVBRCU1
CLIENT_SECRET:MXMXV5RKVWDXJVT3Y0PGCXGF2GYIHSTIJJ1F32ENDGUQXNA0


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=new_df['Neighborhood'],
                                   latitudes=new_df['Latitude'],
                                   longitudes=new_df['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17


In [46]:
toronto_venues["Venue Category"].unique()

array(['Park', 'Fast Food Restaurant', 'Food & Drink Shop',
       'Hockey Arena', 'Coffee Shop', 'Portuguese Restaurant',
       'Pizza Place', 'Bakery', 'Breakfast Spot', 'Distribution Center',
       'Spa', 'Restaurant', 'Gym / Fitness Center', 'Historic Site',
       'Farmers Market', 'Chocolate Shop', 'Pub', 'Performing Arts Venue',
       'Dessert Shop', 'French Restaurant', 'Yoga Studio', 'Café',
       'Theater', 'Event Space', 'Ice Cream Shop', 'Shoe Store',
       'Art Gallery', 'Cosmetics Shop', 'Bank', 'Electronics Store',
       'Beer Store', 'Health Food Store', 'Antique Shop', 'Boutique',
       'Furniture / Home Store', 'Vietnamese Restaurant',
       'Clothing Store', 'Accessories Store', "Women's Store",
       'Miscellaneous Shop', 'Sushi Restaurant', 'Italian Restaurant',
       'Creperie', 'Mexican Restaurant', 'Beer Bar',
       'Arts & Crafts Store', 'Burrito Place', 'Hobby Shop', 'Diner',
       'Fried Chicken Joint', 'Discount Store', 'Wings Joint',
       'Smo

In [21]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2               Skating Rink   0.2
3  Latin American Restaurant   0.2
4             Clothing Store   0.2


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.22
1  Athletics & Sports  0.11
2         Coffee Shop  0.11
3                 Pub  0.11
4        Dance Studio  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0              Bank  0.10
1       Coffee Shop  0.10
2    Sandwich Place  0.05
3  Sushi Restaurant  0.05
4       Supermarket  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3

                    venue  freq
0             Coffee Shop  0.09
1            Burger Joint  0.06
2                    Bank  0.06
3  Furniture / Home Store  0.06
4     Sporting Goods Shop  0.06


----Little Portugal, Trinity----
                           venue  freq
0                            Bar  0.11
1               Asian Restaurant  0.07
2  Vegetarian / Vegan Restaurant  0.04
3                    Coffee Shop  0.04
4                     Restaurant  0.04


----Malvern, Rouge----
                             venue  freq
0             Fast Food Restaurant   1.0
1                    Metro Station   0.0
2  Molecular Gastronomy Restaurant   0.0
3       Modern European Restaurant   0.0
4                Mobile Phone Shop   0.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
         venue  freq
0       Bakery  0.25
1   Playground  0.25
2         Park  0.25
3  Coffee Shop  0.25
4  Yoga Studio  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South We

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [87]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Athletics & Sports,Pub,Dance Studio,Sandwich Place,Diner,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Frozen Yogurt Shop,Supermarket,Ice Cream Shop,Sushi Restaurant,Shopping Mall,Middle Eastern Restaurant,Restaurant,Mobile Phone Shop
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Juice Bar,Thai Restaurant,Liquor Store,Indian Restaurant,Butcher,Pub
5,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Beer Bar,Bakery,Restaurant,Basketball Stadium,Beach
6,"Birch Cliff, Cliffside West",General Entertainment,College Stadium,Café,Skating Rink,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
7,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Climbing Gym,Nightclub,Burrito Place,Italian Restaurant,Restaurant,Intersection
8,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Smoke Shop,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Bar,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Boutique,Airport Food Court


In [88]:
label_list = []
for i in range(0,94):
    first_venue = neighborhoods_venues_sorted.loc[i,"1st Most Common Venue"]
    second_venue = neighborhoods_venues_sorted.loc[i,"2nd Most Common Venue"]
    third_venue = neighborhoods_venues_sorted.loc[i,"3rd Most Common Venue"]
    fv_r_true = False
    sv_r_true = False
    tv_r_true = False
    fv_r_true = "Restaurant" in first_venue 
    sv_r_true = "Restaurant" in second_venue
    tv_r_true = "Restaurant" in third_venue

    low_dens_list = ["Park","River","Trail"]
    low_dens_true = False

    for i in low_dens_list:
        if low_dens_true == False:
            low_dens_true = i in first_venue
            
        if low_dens_true == False:
            low_dens_true = i in second_venue
            
        if low_dens_true == False:
            low_dens_true = i in third_venue

#1 means ideal location, 2 means saturated, 3 means low-density area 
# For 1 need to confirm if venues actually are high traffic venues
# For 2 need to confirm what kind of restaurants and other venues are nearby to determine if actually saturated
# For 3 need to confirm if actually low-density

    label = 1        
    if fv_r_true or sv_r_true or tv_r_true:
        label = 2
    elif low_dens_true:
        label = 3
    
    
    label_list.append(label)

neighborhoods_venues_sorted.insert(1,"Label",label_list)
neighborhoods_venues_sorted

,Neighborhood,Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,2,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,"Alderwood, Long Branch",1,Pizza Place,Coffee Shop,Gym,Pharmacy,Athletics & Sports,Pub,Dance Studio,Sandwich Place,Diner,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",1,Bank,Coffee Shop,Frozen Yogurt Shop,Supermarket,Ice Cream Shop,Sushi Restaurant,Shopping Mall,Middle Eastern Restaurant,Restaurant,Mobile Phone Shop
3,Bayview Village,2,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",2,Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Juice Bar,Thai Restaurant,Liquor Store,Indian Restaurant,Butcher,Pub
5,Berczy Park,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Beer Bar,Bakery,Restaurant,Basketball Stadium,Beach
6,"Birch Cliff, Cliffside West",1,General Entertainment,College Stadium,Café,Skating Rink,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
7,"Brockton, Parkdale Village, Exhibition Place",1,Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Climbing Gym,Nightclub,Burrito Place,Italian Restaurant,Restaurant,Intersection
8,"Business reply mail Processing Centre, South C...",1,Light Rail Station,Yoga Studio,Garden Center,Smoke Shop,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop
9,"CN Tower, King and Spadina, Railway Lands, Har...",1,Airport Service,Airport Terminal,Airport Lounge,Bar,Plane,Rental Car Location,Coffee Shop,Boat or Ferry,Boutique,Airport Food Court


In [89]:
toronto_merged = new_df

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Food & Drink Shop,Park,Fast Food Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Hockey Arena,Portuguese Restaurant,Coffee Shop,Pizza Place,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Historic Site,Performing Arts Venue,Beer Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Coffee Shop,Event Space,Miscellaneous Shop,Vietnamese Restaurant,Boutique,Accessories Store,Eastern European Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Sushi Restaurant,Creperie,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park,College Auditorium
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
7,M3B,North York,Don Mills,43.745906,-79.352188,2.0,Beer Store,Asian Restaurant,Gym,Restaurant,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Supermarket,Sporting Goods Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Gym / Fitness Center,Bank,Pet Store,Pharmacy,Fast Food Restaurant,Intersection,Athletics & Sports,Gastropub,Discount Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Clothing Store,Coffee Shop,Bubble Tea Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Ramen Restaurant


In [92]:
toronto_merged.dropna(axis = 0 , inplace = True)
toronto_merged['Label'].isna().sum()

0

In [95]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lon, poi, label in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Label']):
    
    if label == 1:
        label = folium.Popup(str(poi) + " - Ideal Location", parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='green',
            fill=True,
            fill_color='#42d479',
            fill_opacity=0.7).add_to(map_clusters)
    elif label == 2:
        label = folium.Popup(str(poi) + " - Saturated", parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='orange',
            fill=True,
            fill_color='#ff5733',
            fill_opacity=0.7).add_to(map_clusters)
    elif label == 3:
        label = folium.Popup(str(poi) + " - Low-Density", parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [97]:
toronto_merged.loc[toronto_merged['Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,1.0,Clothing Store,Furniture / Home Store,Women's Store,Coffee Shop,Event Space,Miscellaneous Shop,Vietnamese Restaurant,Boutique,Accessories Store,Eastern European Restaurant
8,East York,1.0,Pizza Place,Gym / Fitness Center,Bank,Pet Store,Pharmacy,Fast Food Restaurant,Intersection,Athletics & Sports,Gastropub,Discount Store
9,Downtown Toronto,1.0,Clothing Store,Coffee Shop,Bubble Tea Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Ramen Restaurant
14,East York,1.0,Curling Ice,Athletics & Sports,Beer Store,Spa,Diner,Skating Rink,Park,Pharmacy,Comfort Food Restaurant,Dessert Shop
17,Etobicoke,1.0,Coffee Shop,Pizza Place,Beer Store,Shopping Plaza,Liquor Store,Convenience Store,Café,Pharmacy,Pet Store,Comfort Food Restaurant
18,Scarborough,1.0,Medical Center,Electronics Store,Rental Car Location,Breakfast Spot,Mexican Restaurant,Bank,Intersection,Distribution Center,Diner,Discount Store
23,East York,1.0,Coffee Shop,Bank,Furniture / Home Store,Burger Joint,Sporting Goods Shop,Bike Shop,Fish & Chips Shop,Sandwich Place,Department Store,Dessert Shop
28,North York,1.0,Bank,Coffee Shop,Frozen Yogurt Shop,Supermarket,Ice Cream Shop,Sushi Restaurant,Shopping Mall,Middle Eastern Restaurant,Restaurant,Mobile Phone Shop
31,West Toronto,1.0,Bakery,Pharmacy,Bank,Brewery,Park,Grocery Store,Music Venue,Café,Bar,Middle Eastern Restaurant
32,Scarborough,1.0,Playground,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


In [99]:
toronto_merged.loc[toronto_merged['Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Food & Drink Shop,Park,Fast Food Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,North York,2.0,Hockey Arena,Portuguese Restaurant,Coffee Shop,Pizza Place,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,Downtown Toronto,2.0,Coffee Shop,Sushi Restaurant,Creperie,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park,College Auditorium
6,Scarborough,2.0,Fast Food Restaurant,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
7,North York,2.0,Beer Store,Asian Restaurant,Gym,Restaurant,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Supermarket,Sporting Goods Shop
10,North York,2.0,Pizza Place,Japanese Restaurant,Asian Restaurant,Pub,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
12,Scarborough,2.0,Bar,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Fast Food Restaurant
13,North York,2.0,Beer Store,Asian Restaurant,Gym,Restaurant,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Supermarket,Sporting Goods Shop
15,Downtown Toronto,2.0,Café,Coffee Shop,Restaurant,Cocktail Bar,American Restaurant,Gastropub,Department Store,Lingerie Store,Gym,Theater
20,Downtown Toronto,2.0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Beer Bar,Bakery,Restaurant,Basketball Stadium,Beach


In [100]:
toronto_merged.loc[toronto_merged['Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,3.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Historic Site,Performing Arts Venue,Beer Store
16,York,3.0,Trail,Hockey Arena,Field,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
19,East Toronto,3.0,Trail,Health Food Store,Pub,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
21,York,3.0,Park,Pool,Women's Store,Greek Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
25,Downtown Toronto,3.0,Grocery Store,Café,Park,Italian Restaurant,Baby Store,Athletics & Sports,Candy Store,Diner,Restaurant,Coffee Shop
35,East York,3.0,Convenience Store,Park,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
40,North York,3.0,Grocery Store,Park,Shopping Mall,Snack Place,Baseball Field,Food Truck,Liquor Store,Business Service,Athletics & Sports,Home Service
44,Scarborough,3.0,Bakery,Park,Metro Station,Soccer Field,Bus Line,Intersection,Bus Station,Ice Cream Shop,Convention Center,Concert Hall
46,North York,3.0,Grocery Store,Park,Shopping Mall,Snack Place,Baseball Field,Food Truck,Liquor Store,Business Service,Athletics & Sports,Home Service
49,North York,3.0,Park,Construction & Landscaping,Bakery,Basketball Court,Electronics Store,Eastern European Restaurant,Drugstore,Ethiopian Restaurant,Donut Shop,Department Store
